In [2]:
import requests,time,datetime
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [4]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [5]:
df = df_hsA

In [3]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2019')

ss.getStockDataByType('E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/')
# ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


正在获取：沪深A股...请勿关闭浏览器
第206页-第17条...

Traceback (most recent call last):
  File "E:\wangtao\PythonWorkSpace\DataAnalysis\StockAnalyze\utils\StockSpider.py", line 103, in getStockDataByType
    company = browser.find_element_by_xpath('//*[@id="table_wrapper-table"]/tbody/tr['+str(j+1)+']/td[3]/a')
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath

数据下载完毕，已保存到E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/沪深A股.xls


In [6]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [7]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})
df_2019 = pd.read_excel("data/2019年报.xls",converters= {u'stock_code':str})

In [15]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [16]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [17]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [18]:
df_target.shape

(64, 19)

In [22]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_S']) / x['PE_M']),axis=1)
# df_target = df_target[df_target['pe_rate'] < 0.6]

In [23]:
df_target.shape

(59, 21)

In [24]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
2738,600388,龙净环保,9.85,-0.51%,1.26,1.42%,机械设备 — 专用设备,1.93,12.44,39.008,0.06元,0.68亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降35.1%\t\...,1.18%,21.77%,5.13元,10.69亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,-0.10元,21.970,0.566227
1444,000963,华东医药,27.45,0.73%,0.93,1.13%,医药生物 — 化学制药,3.54,16.95,10.390,0.66元,11.47亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长26.46%\...,8.89%,40.73%,7.70元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.12元,27.210,0.622933
2834,600398,海澜之家,6.66,-0.60%,0.67,0.29%,纺织服装 — 服装家纺,2.24,9.02,24.505,0.07元,2.95亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降75.59%\t...,2.15%,34.48%,2.99元,43.20亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.22元,13.560,0.665192
2486,002262,恩华药业,18.48,-0.27%,0.53,1.64%,医药生物 — 化学制药,4.84,28.47,26.763,0.35元,3.53亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长9.64%\t\...,9.28%,75.68%,3.83元,10.19亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,0.40元,38.965,0.730656
1203,603585,苏利股份,19.40,1.09%,1.18,1.00%,化工 — 化学制品,1.73,11.39,22.070,0.22元,0.39亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降51.02%\t...,1.97%,31.03%,11.12元,1.80亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,NaN,0.03元,13.820,0.824168
3519,000002,万 科Ａ,27.70,-1.81%,0.79,0.87%,房地产 — 房地产开发Ⅱ,1.63,8.28,64.372,0.11元,12.49亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长11.49%\...,0.66%,31.28%,16.92元,116.18亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,-0.26元,9.920,0.834677
675,002912,中新赛克,87.03,2.45%,0.94,3.42%,计算机 — 计算机设备,9.59,50.23,108.898,0.18元,0.32亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长616.77%\...,2.09%,74.48%,8.86元,1.74亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,-0.32元,58.480,0.858926
3465,601318,中国平安,79.75,-1.66%,1.01,0.78%,非银金融 — 保险,2.11,9.76,13.986,1.47元,260.63亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降42.74%...,3.80%,未公布,37.76元,182.80亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,7.24元,11.055,0.882858
2133,601019,山东出版,6.79,0.00%,1.08,1.44%,传媒 — 文化传媒,1.31,9.18,25.611,0.07元,1.38亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降36.41%\t...,1.28%,35.86%,5.20元,20.87亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,NaN,-0.31元,10.390,0.883542
1561,603983,丸美股份,76.36,0.58%,1.36,2.10%,化工 — 化学制品,10.91,59.13,64.042,0.30元,1.19亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降1.14%\t\...,4.35%,65.51%,6.96元,4.01亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,-0.05元,63.740,0.927675
